In [1]:
import timeit

In [2]:
a=1
b=2

In [3]:
total_time = timeit.timeit('a+b', globals=locals(), number=1_000_000)

In [8]:
total_time

0.058483081000005654

In [9]:
%timeit a+b

50.2 ns ± 0.957 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [10]:
%timeit 1+2

10.4 ns ± 0.0633 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [12]:
t_var = %timeit -o a+b

51.7 ns ± 1.02 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [13]:
t_lit = %timeit -o 1+2

10.6 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [14]:
t_var.average/t_lit.average

4.874937952696691

In [15]:
%%timeit
a=1
b=2
a+b

47.2 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#Profiling

In [20]:
# file profile_me.py

"""Example to be profiled.
"""
import sys
import time

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)


def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()


def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


In [21]:
import cProfile

In [24]:
profiler = cProfile.Profile()

In [25]:
profiler.runcall(use_fast)

In [26]:
profiler.print_stats()

         202 function calls in 0.126 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.001    0.000    0.125    0.001 <ipython-input-20-7d760d70ba0b>:12(fast)
        1    0.001    0.001    0.126    0.126 <ipython-input-20-7d760d70ba0b>:24(use_fast)
      100    0.124    0.001    0.124    0.001 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [27]:
profiler2 = cProfile.Profile()

In [28]:
profiler2.runcall(use_slow)

In [29]:
profiler2.print_stats()

         202 function calls in 10.299 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.002    0.000   10.298    0.103 <ipython-input-20-7d760d70ba0b>:18(slow)
        1    0.001    0.001   10.299   10.299 <ipython-input-20-7d760d70ba0b>:31(use_slow)
      100   10.296    0.103   10.296    0.103 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [30]:
cProfile.run('use_fast()', 'fast.stats')

In [31]:
%ls fast.stats

fast.stats


In [33]:
import pstats

In [34]:
pstats

<module 'pstats' from '/Applications/anaconda3/lib/python3.7/pstats.py'>

In [37]:
#save for later so you can go back and look
stats = pstats.Stats('fast.stats')

In [39]:
stats.print_stats('calls')

Thu May  2 14:08:15 2019    fast.stats

         204 function calls in 0.123 seconds



In [41]:
# find out who calls your function
stats.print_callers('fast')

   Random listing order was used
   List reduced from 6 to 2 due to restriction <'fast'>

Function                                      was called by...
                                                  ncalls  tottime  cumtime
<ipython-input-20-7d760d70ba0b>:12(fast)      <-     100    0.001    0.123  <ipython-input-20-7d760d70ba0b>:24(use_fast)
<ipython-input-20-7d760d70ba0b>:24(use_fast)  <-       1    0.000    0.123  <string>:1(<module>)




In [48]:
stats.print_callees('fast')

   Random listing order was used
   List reduced from 6 to 2 due to restriction <'fast'>

Function                                      called...
                                                  ncalls  tottime  cumtime
<ipython-input-20-7d760d70ba0b>:12(fast)      ->     100    0.122    0.122  {built-in method time.sleep}
<ipython-input-20-7d760d70ba0b>:24(use_fast)  ->     100    0.001    0.123  <ipython-input-20-7d760d70ba0b>:12(fast)




In [49]:
### With Jupyter

In [50]:
%prun use_fast()

In [53]:
%prun?

In [59]:
stats_slow = %prun -r use_slow()

In [60]:
stats_slow.print_stats()

         204 function calls in 10.235 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100   10.233    0.102   10.233    0.102 {built-in method time.sleep}
      100    0.001    0.000   10.234    0.102 <ipython-input-20-7d760d70ba0b>:18(slow)
        1    0.001    0.001   10.235   10.235 <ipython-input-20-7d760d70ba0b>:31(use_slow)
        1    0.000    0.000   10.235   10.235 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000   10.235   10.235 <string>:1(<module>)




# Wall Clock vs CPU Time

In [66]:
# %load measuring/clock_check.py
# file: measuring/clock_check.py

"""Checking different timing functions.
"""

from __future__ import print_function

import os
import sys
import time
import timeit


if sys.version_info.major < 3:
    range = xrange


def clock_check(duration=1):
    """Check the measured time with different methods.
    """
    start_os_time0 = os.times()[0]
    start_time_clock = time.clock()
    start_default_timer = timeit.default_timer()
    for _ in range(int(1e6)):
        1 + 1
    time.sleep(duration)
    durtation_os_time0 = os.times()[0] - start_os_time0
    durtation_time_clock = time.clock() - start_time_clock
    durtation_default_timer = timeit.default_timer() - start_default_timer
    print('durtation_os_time0:     ', durtation_os_time0)
    print('durtation_time_clock:   ', durtation_time_clock)
    print('durtation_default_timer:', durtation_default_timer)


if __name__ == '__main__':
    clock_check()


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


durtation_os_time0:      0.030000000000001137
durtation_time_clock:    0.03193300000000221
durtation_default_timer: 1.0312501219996193


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [69]:
# %load measuring/cpu_time.py
# file: cpu_time.py

"""Measuring CPU time instead of wall clock time.
"""

import cProfile
import os
import sys
import time

# Make it work with Python 2 and Python 3.
if sys.version_info.major < 3:
    range = xrange


def cpu_time():
    """Function for cpu time. Os dependent.
    """
    if sys.platform == 'win32':
        return os.times()[0]
    else:
        return time.clock()


def sleep():
    """Wait 2 seconds.
    """
    time.sleep(2)


def count():
    """100 million loops.
    """
    for _ in range(int(1e8)):
        1 + 1


def test():
    """Run functions
    """
    sleep()
    count()


def profile():
    """Profile with wall clock and cpu time.
    """
    profiler = cProfile.Profile()
    profiler.run('test()')
    profiler.print_stats()

    profiler = cProfile.Profile(cpu_time)
    profiler.run('test()')
    profiler.print_stats()

if __name__ == '__main__':
    profile()


         7 function calls in 5.030 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.005    2.005 <ipython-input-69-6870ca2d00ae>:26(sleep)
        1    3.025    3.025    3.025    3.025 <ipython-input-69-6870ca2d00ae>:32(count)
        1    0.000    0.000    5.030    5.030 <ipython-input-69-6870ca2d00ae>:39(test)
        1    0.000    0.000    5.030    5.030 <string>:1(<module>)
        1    0.000    0.000    5.030    5.030 {built-in method builtins.exec}
        1    2.005    2.005    2.005    2.005 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


         7 function calls in 3.083 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.008    0.008 <ipython-input-69-6870ca2d00ae>:26(sleep)
        1    3.075    3.075    3.075    3.075 <ipython-input-69-6870ca2d00ae>:32(count)
        1    0.000    0.000    3.083    3.083 <ipython-input-69-6870ca2d00ae>:39(test)
        1    0.000    0.000    3.083    3.083 <string>:1(<module>)
        1    0.000    0.000    3.083    3.083 {built-in method builtins.exec}
        1    0.008    0.008    0.008    0.008 {built-in method time.sleep}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [70]:
import time

In [75]:
#Wall time
time.perf_counter

<function time.perf_counter>

In [76]:
#CPU time
time.process_time

<function time.process_time>

In [79]:
%ls

Timing Practice.ipynb       measuring/
algorithms/                 pi/
caching/                    profiling.pdf
datastructure/              profiling_presentation.pdf
fast.stats


In [80]:
%cd pi

/Users/jared/Documents/GitHub/PyCon2019_FasterPythonWorkshop/optimizing/pi


In [83]:
# %load simple_pi.py
# file: simple_pi.py

"""Calculating pi with Monte Carlo.
"""

from __future__ import print_function

import math
import random
import sys


if sys.version_info[0] < 3:
    range = xrange


def pi_plain(total):
    """Calculate pi with `total` hits.
    """
    count_inside = 0
    for _ in range(total):
        x = random.random()
        y = random.random()
        dist = math.sqrt(x * x + y * y)
        if dist < 1:
            count_inside += 1
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Check if it works.
        """
        n = int(1e6)
        print('pi:', pi_plain(n))

    test()


pi: 3.14132


In [85]:
# %load numpy_pi.py
# file: numpy_pi.py
"""Calculating pi with Monte Carlo Method and NumPy.
"""

from __future__ import print_function

import numpy                                                   #1


def pi_numpy(total):                                           #2
    """Compute pi.
    """
    x = numpy.random.rand(total)                               #3
    y = numpy.random.rand(total)                               #4
    dist = numpy.sqrt(x * x + y * y)                           #5
    count_inside = len(dist[dist < 1])                         #6
    return 4.0 * count_inside / total

if __name__ == '__main__':

    def test():
        """Time the execution.
        """
        import timeit
        start = timeit.default_timer()
        pi_numpy(int(1e6))
        print('run time', timeit.default_timer() - start)
    test()


run time 0.42995082800007367


In [95]:
%load_ext snakeviz

ModuleNotFoundError: No module named 'snakeviz'

In [115]:
# %load profile_me_use_line_profiler.py
# file profile_me_use_line_profiler.py

"""Example to be profiled.
"""

import time
import sys

if sys.version_info.major < 3:
    range = xrange


def fast():
    """Wait 0.001 seconds.
    """
    time.sleep(1e-3)


def slow():
    """Wait 0.1 seconds.
    """
    time.sleep(0.1)

@profile
def use_fast():
    """Call `fast` 100 times.
    """
    for _ in range(100):
        fast()

@profile
def use_slow():
    """Call `slow` 100 times.
    """
    for _ in range(100):
        slow()


if __name__ == '__main__':
    use_fast()
    use_slow()


TypeError: profile() takes 0 positional arguments but 1 was given

In [117]:
%load_ext line_profiler

ModuleNotFoundError: No module named 'line_profiler'